In [4]:
from models.ffnn import BrainFFNN
from models.vgg import BrainVGG

In [5]:
ffnn = BrainFFNN()

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [6]:
ffnn.train()

Epoch 1/10


2024-03-02 16:26:03.106979: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f26ec7584a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-02 16:26:03.107021: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2024-03-02 16:26:03.112531: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-02 16:26:03.128468: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1709414763.202801    7497 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


179/179 [==============================] - 6s 26ms/step - loss: 982.9663 - accuracy: 0.5709
Epoch 2/10
179/179 [==============================] - 5s 25ms/step - loss: 217.4289 - accuracy: 0.7031
Epoch 3/10
179/179 [==============================] - 5s 26ms/step - loss: 161.4192 - accuracy: 0.7208
Epoch 4/10
179/179 [==============================] - 5s 25ms/step - loss: 83.1845 - accuracy: 0.7654
Epoch 5/10
179/179 [==============================] - 5s 26ms/step - loss: 60.7575 - accuracy: 0.7785
Epoch 6/10
179/179 [==============================] - 5s 26ms/step - loss: 24.6072 - accuracy: 0.8276
Epoch 7/10
179/179 [==============================] - 5s 26ms/step - loss: 15.0684 - accuracy: 0.8465
Epoch 8/10
179/179 [==============================] - 5s 25ms/step - loss: 16.2113 - accuracy: 0.8274
Epoch 9/10
179/179 [==============================] - 5s 25ms/step - loss: 12.9150 - accuracy: 0.8466
Epoch 10/10
179/179 [==============================] - 5s 25ms/step - loss: 10.2604 - accu

In [7]:
ffnn.eval()

41/41 [==============================] - 1s 14ms/step - loss: 32.3539 - accuracy: 0.6926


In [8]:
ffnn.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 196608)            0         
                                                                 
 dense_3 (Dense)             (None, 256)               50331904  
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 4)                 516       
                                                                 
Total params: 50365316 (192.13 MB)
Trainable params: 50365316 (192.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
ffnn.save()

In [10]:
vgg = BrainVGG()

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [11]:
vgg.train()

Epoch 1/10


2024-03-02 16:26:54.737188: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-02 16:26:54.891102: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-02 16:26:54.908031: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 528.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-02 16:26:54.908105: W tensorflow/core/kernels/gpu_utils.c

ResourceExhaustedError: Graph execution error:

Detected at node vgg19/block2_conv2/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/caesar/TumorML/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/caesar/TumorML/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/caesar/TumorML/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/caesar/TumorML/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/caesar/TumorML/.venv/lib/python3.10/site-

In [ ]:
vgg.eval()

In [ ]:
vgg.model.summary()

In [ ]:
vgg.save()